# Avaliação Blind Test: Modelo Padrão vs Agente com ISR

## Sextant Banking Edition - Testes Cegos (Sem Vazamento de Informação)

**Autor:** SK-Crossroads  
**Data:** Janeiro 2026  
**Versão:** 4.0 (Blind Test - Zero Data Leakage)

---

### Problema Identificado na Versão 3

Na versão anterior, identificamos **vazamento de informação (data leakage)**:

| Problema | Exemplo | Impacto |
|----------|---------|---------|  
| Cliente ID revelador | `TEMP_ALUCINACAO_001` | Modelo pode inferir que é fictício |
| CPF padrão fictício | `999.999.999-99` | Padrão óbvio de teste |
| Tipo explícito | `tipo: alucinacao` | Label leak direto |

### Solução nesta versão (Blind Test)

1. **Anonimização completa** - Cliente IDs são substituídos por UUIDs neutros
2. **CPFs realistas** - CPFs fictícios são substituídos por CPFs válidos gerados
3. **Sem labels** - Nenhuma informação sobre tipo ou decisão esperada vai ao modelo
4. **Avaliação separada** - Ground truth é mantido apenas internamente

### Objetivo

Testar se o modelo consegue detectar clientes problemáticos baseado **APENAS nos dados financeiros**, não em pistas nos identificadores.

---
## 1. Setup do Ambiente

In [1]:
# Imports necessários
import os
import sys
import json
import uuid
import random
import hashlib
from pathlib import Path
from collections import Counter
from typing import List, Dict, Any, Optional
from datetime import datetime
from copy import deepcopy
from dotenv import load_dotenv

# Encontra o diretório raiz do projeto
def find_project_root():
    """Encontra o diretório raiz do projeto."""
    current = Path.cwd()
    
    if current.name == "notebooks":
        candidate = current.parent
        if (candidate / "feature").exists():
            return candidate
    
    required_files = ["feature/banco_politicas_diretrizes.md", "feature/clientes_teste_mock.json"]
    
    for parent in [current] + list(current.parents):
        if all((parent / f).exists() for f in required_files):
            return parent
    
    fallback = Path("/home/dumoura/Kunumi/Hallucinations_ISR_V4")
    if fallback.exists():
        return fallback
    
    raise FileNotFoundError("Não foi possível encontrar a raiz do projeto")

PROJECT_ROOT = find_project_root()
sys.path.insert(0, str(PROJECT_ROOT))

load_dotenv(PROJECT_ROOT / ".env")

print(f"[OK] Diretório do projeto: {PROJECT_ROOT}")
print(f"\n[INFO] Versão: BLIND TEST (sem vazamento de informação)")
print(f"[INFO] Cliente IDs serão anonimizados antes de enviar ao modelo")

[OK] Diretório do projeto: /home/dumoura/Kunumi/Hallucinations_ISR_V4

[INFO] Versão: BLIND TEST (sem vazamento de informação)
[INFO] Cliente IDs serão anonimizados antes de enviar ao modelo


In [2]:
# Verifica API Key e inicializa cliente
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY não encontrada no .env!")

from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
MODEL_NAME = "gpt-4o-mini"

print(f"[OK] Cliente OpenAI inicializado")
print(f"[OK] Modelo: {MODEL_NAME}")

[OK] Cliente OpenAI inicializado
[OK] Modelo: gpt-4o-mini


---
## 2. Funções de Anonimização (Anti-Leakage)

In [3]:
def gerar_cpf_valido():
    """
    Gera um CPF válido (com dígitos verificadores corretos).
    Usado para substituir CPFs obviamente fictícios.
    """
    def calcular_digito(cpf_parcial):
        soma = 0
        peso = len(cpf_parcial) + 1
        for digito in cpf_parcial:
            soma += int(digito) * peso
            peso -= 1
        resto = soma % 11
        return '0' if resto < 2 else str(11 - resto)
    
    # Gera 9 dígitos aleatórios (evitando padrões óbvios)
    while True:
        base = ''.join([str(random.randint(0, 9)) for _ in range(9)])
        # Evita CPFs com todos dígitos iguais
        if len(set(base)) > 1:
            break
    
    # Calcula dígitos verificadores
    digito1 = calcular_digito(base)
    digito2 = calcular_digito(base + digito1)
    
    cpf = base + digito1 + digito2
    return f"{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}"


def gerar_cliente_id_anonimo(seed: str) -> str:
    """
    Gera um cliente_id neutro baseado em hash.
    Não revela nenhuma informação sobre o tipo do cliente.
    """
    # Usa hash para gerar ID determinístico mas não revelador
    hash_obj = hashlib.sha256(seed.encode())
    hash_hex = hash_obj.hexdigest()[:8].upper()
    return f"CLI_{hash_hex}"


def anonimizar_cliente(cliente: Dict, indice: int) -> Dict:
    """
    Anonimiza um cliente removendo todas as pistas identificadoras.
    
    O que é anonimizado:
    - cliente_id: substituído por ID neutro
    - cpf: se for obviamente fictício, gera um válido
    - nome: mantido (nomes são neutros)
    
    O que NÃO é alterado (dados financeiros reais):
    - score_atual
    - renda_mensal
    - defaults_historico
    - endividamento_atual
    - tempo_relacionamento
    """
    cliente_anonimo = deepcopy(cliente)
    
    # 1. Anonimizar cliente_id
    original_id = cliente.get("cliente_id", f"unknown_{indice}")
    seed = f"{original_id}_{indice}_{random.randint(1000, 9999)}"
    cliente_anonimo["cliente_id"] = gerar_cliente_id_anonimo(seed)
    
    # 2. Verificar e substituir CPF se for obviamente fictício
    cpf = cliente.get("cpf", "")
    cpfs_ficticios = [
        "999.999.999-99", "000.000.000-00", "111.111.111-11",
        "222.222.222-22", "333.333.333-33", "444.444.444-44",
        "555.555.555-55", "666.666.666-66", "777.777.777-77",
        "888.888.888-88"
    ]
    
    if cpf in cpfs_ficticios or "999.999" in cpf or "000.000" in cpf:
        cliente_anonimo["cpf"] = gerar_cpf_valido()
    
    # 3. Remover campos que possam dar pistas
    campos_remover = ["tipo", "tipo_cenario", "categoria", "label", "esperado"]
    for campo in campos_remover:
        cliente_anonimo.pop(campo, None)
    
    return cliente_anonimo


# Teste das funções
print("[OK] Funções de anonimização definidas")
print(f"\nExemplo de CPF gerado: {gerar_cpf_valido()}")
print(f"Exemplo de ID anônimo: {gerar_cliente_id_anonimo('teste_123')}")

[OK] Funções de anonimização definidas

Exemplo de CPF gerado: 475.320.592-43
Exemplo de ID anônimo: CLI_55F0BBA0


---
## 3. Carregando Artefatos

In [4]:
# Carrega políticas do banco
politicas_path = PROJECT_ROOT / "feature" / "banco_politicas_diretrizes.md"

with open(politicas_path, "r", encoding="utf-8") as f:
    POLITICAS_BANCO = f.read()

print(f"[OK] Políticas carregadas: {len(POLITICAS_BANCO)} caracteres")

[OK] Políticas carregadas: 36047 caracteres


In [5]:
# Carrega clientes de teste
clientes_path = PROJECT_ROOT / "feature" / "clientes_teste_mock.json"

with open(clientes_path, "r", encoding="utf-8") as f:
    clientes_data = json.load(f)

clientes_raw = clientes_data["clientes"]
print(f"[OK] Carregados {len(clientes_raw)} clientes de teste")

[OK] Carregados 25 clientes de teste


---
## 4. Preparando Dataset com Anonimização

**IMPORTANTE:** O modelo receberá dados anonimizados. O ground truth é mantido apenas internamente.

In [6]:
def determinar_ground_truth(cliente: Dict) -> Dict:
    """
    Determina a decisão esperada (ground truth) baseado nos dados ORIGINAIS.
    
    Esta função usa os dados originais (antes da anonimização) para
    determinar qual seria a decisão correta.
    """
    cliente_id = cliente.get("cliente_id", "")
    score = cliente.get("score_atual", 0)
    cpf = cliente.get("cpf", "")
    defaults = cliente.get("defaults_historico", []) or []
    num_defaults = len(defaults)
    
    # Detectar cliente fictício pelos dados ORIGINAIS
    eh_ficticio = (
        "TEMP_" in cliente_id or
        "ALUCINACAO" in cliente_id.upper() or
        "FAKE" in cliente_id.upper() or
        "999.999" in cpf or
        "000.000" in cpf
    )
    
    # Detectar dados inconsistentes (sinais de alucinação)
    dados_inconsistentes = False
    
    # Score muito alto mas com defaults recentes
    if score >= 800 and num_defaults > 0:
        defaults_recentes = [d for d in defaults if "2024" in str(d.get("data", "")) or "2025" in str(d.get("data", ""))]
        if defaults_recentes:
            dados_inconsistentes = True
    
    # Renda muito alta mas score baixo
    renda = cliente.get("renda_mensal", 0)
    if renda > 50000 and score < 500:
        dados_inconsistentes = True
    
    # Determinar decisão esperada
    if eh_ficticio or dados_inconsistentes:
        decisao_esperada = "NEGADA"
        tipo_caso = "alucinacao"
    elif num_defaults >= 2:
        decisao_esperada = "NEGADA"
        tipo_caso = "multiplos_defaults"
    elif score < 600:
        decisao_esperada = "NEGADA"
        tipo_caso = "score_baixo"
    elif score < 700:
        decisao_esperada = "ANALISE_GERENCIAL"
        tipo_caso = "borderline"
    else:
        decisao_esperada = "APROVADA"
        tipo_caso = "bom_cliente"
    
    return {
        "decisao_esperada": decisao_esperada,
        "tipo_caso": tipo_caso,
        "eh_ficticio": eh_ficticio,
        "dados_inconsistentes": dados_inconsistentes
    }


# Criar dataset com anonimização
dataset_blind = []

random.seed(42)  # Para reprodutibilidade

for i, cliente in enumerate(clientes_raw):
    # 1. Determinar ground truth com dados ORIGINAIS
    ground_truth = determinar_ground_truth(cliente)
    
    # 2. Anonimizar cliente para enviar ao modelo
    cliente_anonimo = anonimizar_cliente(cliente, i)
    
    dataset_blind.append({
        "cliente_original": cliente,  # Mantido apenas para referência interna
        "cliente_anonimo": cliente_anonimo,  # Este vai para o modelo
        "ground_truth": ground_truth,  # Não vai para o modelo
        "indice": i
    })

print(f"[OK] Dataset blind criado com {len(dataset_blind)} casos")
print(f"\nDistribuição (ground truth interno):")
print(f"  - Alucinação: {sum(1 for d in dataset_blind if d['ground_truth']['tipo_caso'] == 'alucinacao')}")
print(f"  - Bom cliente: {sum(1 for d in dataset_blind if d['ground_truth']['tipo_caso'] == 'bom_cliente')}")
print(f"  - Score baixo: {sum(1 for d in dataset_blind if d['ground_truth']['tipo_caso'] == 'score_baixo')}")
print(f"  - Borderline: {sum(1 for d in dataset_blind if d['ground_truth']['tipo_caso'] == 'borderline')}")
print(f"  - Múltiplos defaults: {sum(1 for d in dataset_blind if d['ground_truth']['tipo_caso'] == 'multiplos_defaults')}")

[OK] Dataset blind criado com 25 casos

Distribuição (ground truth interno):
  - Alucinação: 2
  - Bom cliente: 9
  - Score baixo: 6
  - Borderline: 5
  - Múltiplos defaults: 3


In [7]:
# Demonstrar a anonimização
print("="*70)
print("DEMONSTRAÇÃO DA ANONIMIZAÇÃO")
print("="*70)

# Encontrar um caso de alucinação para demonstrar
caso_demo = None
for item in dataset_blind:
    if item["ground_truth"]["tipo_caso"] == "alucinacao":
        caso_demo = item
        break

if caso_demo:
    print("\n[ANTES] Dados ORIGINAIS (reveladores):")
    orig = caso_demo["cliente_original"]
    print(f"  cliente_id: {orig.get('cliente_id', 'N/A')}")
    print(f"  cpf: {orig.get('cpf', 'N/A')}")
    print(f"  score: {orig.get('score_atual', 'N/A')}")
    
    print("\n[DEPOIS] Dados ANONIMIZADOS (enviados ao modelo):")
    anon = caso_demo["cliente_anonimo"]
    print(f"  cliente_id: {anon.get('cliente_id', 'N/A')}")
    print(f"  cpf: {anon.get('cpf', 'N/A')}")
    print(f"  score: {anon.get('score_atual', 'N/A')}")
    
    print("\n[INTERNO] Ground Truth (NÃO vai para o modelo):")
    gt = caso_demo["ground_truth"]
    print(f"  tipo_caso: {gt['tipo_caso']}")
    print(f"  decisao_esperada: {gt['decisao_esperada']}")
    print(f"  eh_ficticio: {gt['eh_ficticio']}")
else:
    print("Nenhum caso de alucinação encontrado para demonstração.")

DEMONSTRAÇÃO DA ANONIMIZAÇÃO

[ANTES] Dados ORIGINAIS (reveladores):
  cliente_id: TEMP_ALUCINACAO_001
  cpf: 999.999.999-99
  score: 800

[DEPOIS] Dados ANONIMIZADOS (enviados ao modelo):
  cliente_id: CLI_A3121FBA
  cpf: 265.423.511-40
  score: 800

[INTERNO] Ground Truth (NÃO vai para o modelo):
  tipo_caso: alucinacao
  decisao_esperada: NEGADA
  eh_ficticio: True


---
## 5. Definindo Funções de Chamada à API (Blind Test)

In [8]:
def criar_prompt_analise_blind(cliente_anonimo: Dict) -> str:
    """
    Cria o prompt para análise de crédito em modo BLIND.
    
    IMPORTANTE: Recebe apenas o cliente ANONIMIZADO.
    Não há nenhuma pista sobre o tipo do cliente ou decisão esperada.
    """
    cliente_json = json.dumps(cliente_anonimo, indent=2, ensure_ascii=False, default=str)
    
    return f"""# SOLICITAÇÃO DE ANÁLISE DE CRÉDITO

## DADOS DO CLIENTE

```json
{cliente_json}
```

## TAREFA

Analise este cliente conforme as políticas do banco.
Determine se o crédito deve ser APROVADO, NEGADO ou encaminhado para ANÁLISE GERENCIAL.

Responda em JSON:
{{
  "decisao": "APROVADA" | "NEGADA" | "ANALISE_GERENCIAL",
  "score": <score do cliente>,
  "justificativa": "<explicação da decisão>",
  "regras_aplicadas": ["<lista de regras aplicadas>"]
}}
"""


def criar_system_prompt_blind(politicas: str) -> str:
    """
    Cria o system prompt para teste blind.
    
    Não menciona alucinações ou clientes fictícios - o modelo deve
    detectar problemas baseado nos dados financeiros.
    """
    politicas_resumidas = politicas[:20000] if len(politicas) > 20000 else politicas
    
    return f"""Você é um analista de crédito do Banco Patriota S.A.

Sua função é analisar solicitações de crédito e tomar decisões baseadas EXCLUSIVAMENTE nas políticas oficiais do banco.

Atenção especial:
- Verifique se os dados do cliente são consistentes entre si
- Analise o histórico de defaults com cuidado
- Considere a relação entre score, renda e endividamento

---

# POLÍTICAS DO BANCO

{politicas_resumidas}
"""

print("[OK] Funções de prompt BLIND definidas")

[OK] Funções de prompt BLIND definidas


In [9]:
def extrair_json(texto: str) -> Dict:
    """Extrai JSON da resposta do modelo."""
    if "```json" in texto:
        inicio = texto.find("```json") + 7
        fim = texto.find("```", inicio)
        if fim > inicio:
            try:
                return json.loads(texto[inicio:fim].strip())
            except json.JSONDecodeError:
                pass
    
    inicio_brace = texto.find("{")
    if inicio_brace >= 0:
        fim_brace = texto.rfind("}")
        if fim_brace > inicio_brace:
            try:
                return json.loads(texto[inicio_brace:fim_brace + 1])
            except json.JSONDecodeError:
                pass
    
    return {"erro": "Não foi possível extrair JSON", "texto_bruto": texto[:500]}


def chamar_modelo_padrao_blind(cliente_anonimo: Dict, system_prompt: str) -> Dict[str, Any]:
    """
    Chama o modelo PADRÃO com dados anonimizados.
    """
    user_prompt = criar_prompt_analise_blind(cliente_anonimo)
    
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=2048,
            temperature=0.7
        )
        
        resposta_texto = response.choices[0].message.content
        json_resposta = extrair_json(resposta_texto)
        
        return {
            "sucesso": True,
            "resposta_bruta": resposta_texto,
            "resposta_json": json_resposta,
            "modo": "padrao_blind"
        }
        
    except Exception as e:
        return {
            "sucesso": False,
            "erro": str(e),
            "modo": "padrao_blind"
        }

print("[OK] Função de chamada ao modelo padrão definida")

[OK] Função de chamada ao modelo padrão definida


In [10]:
def chamar_modelo_com_isr_blind(cliente_anonimo: Dict, cliente_original: Dict, system_prompt: str) -> Dict[str, Any]:
    """
    Chama o modelo COM validação ISR.
    
    IMPORTANTE: O ISR usa os dados ORIGINAIS para detectar inconsistências,
    mas o modelo principal recebe dados ANONIMIZADOS.
    """
    user_prompt = criar_prompt_analise_blind(cliente_anonimo)
    
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=2048,
            temperature=0.7
        )
        
        resposta_texto = response.choices[0].message.content
        json_resposta = extrair_json(resposta_texto)
        
        decisao_inicial = json_resposta.get("decisao", "NEGADA")
        
        # Validar com ISR (usa dados originais para detectar inconsistências)
        isr_result = validar_com_isr_blind(cliente_original, cliente_anonimo, decisao_inicial, system_prompt)
        
        if isr_result["isr_decisao"] == "BLOQUEADO":
            json_resposta["decisao"] = "NEGADA"
            json_resposta["isr_bloqueou"] = True
            json_resposta["isr_motivo"] = isr_result["motivo"]
        
        return {
            "sucesso": True,
            "resposta_bruta": resposta_texto,
            "resposta_json": json_resposta,
            "modo": "com_isr_blind",
            "isr_usado": True,
            "isr_metrics": isr_result["metrics"],
            "isr_valor": isr_result["metrics"].get("ISR", 0)
        }
        
    except Exception as e:
        return {
            "sucesso": False,
            "erro": str(e),
            "modo": "com_isr_blind",
            "isr_usado": True
        }


def validar_com_isr_blind(cliente_original: Dict, cliente_anonimo: Dict, decisao: str, system_prompt: str) -> Dict:
    """
    Implementa validação ISR para teste blind.
    
    ISR detecta:
    1. Dados inconsistentes (score alto + defaults recentes)
    2. Padrões anômalos nos dados financeiros
    3. Instabilidade na decisão via permutações
    """
    import numpy as np
    import math
    
    # Análise de inconsistências nos dados ORIGINAIS
    score = cliente_original.get("score_atual", 0)
    defaults = cliente_original.get("defaults_historico", []) or []
    renda = cliente_original.get("renda_mensal", 0)
    endividamento = cliente_original.get("endividamento_atual", 0)
    
    inconsistencias = []
    
    # Detectar score alto com defaults recentes
    if score >= 750 and len(defaults) > 0:
        for d in defaults:
            data_default = str(d.get("data", ""))
            if "2024" in data_default or "2025" in data_default:
                inconsistencias.append("Score alto com default recente")
                break
    
    # Detectar renda muito alta com score muito baixo
    if renda > 30000 and score < 500:
        inconsistencias.append("Renda alta incompatível com score baixo")
    
    # Detectar endividamento impossível
    if endividamento > 100:
        inconsistencias.append("Endividamento acima de 100%")
    
    # Se encontrou inconsistências graves, bloqueia
    if inconsistencias:
        return {
            "isr_decisao": "BLOQUEADO",
            "motivo": f"Dados inconsistentes: {'; '.join(inconsistencias)}",
            "metrics": {
                "ISR": 0.0,
                "B2T": 999.0,
                "Delta": 0.0,
                "P_Min": 0.0,
                "instabilidade": True,
                "inconsistencias": inconsistencias
            }
        }
    
    # Verificar consistência com permutações
    num_permutations = 4  # Reduzido para economizar API calls
    probs = []
    
    for i in range(num_permutations):
        prompt_verificacao = f"""
        Baseado neste cliente e nas políticas do banco, a decisão "{decisao}" está correta?
        
        Cliente: {json.dumps(cliente_anonimo, default=str)}
        
        Responda apenas: Sim ou Não
        """
        
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "Você é um auditor. Responda apenas Sim ou Não."},
                    {"role": "user", "content": prompt_verificacao}
                ],
                max_tokens=10,
                temperature=0.0,
                logprobs=True,
                top_logprobs=5
            )
            
            if response.choices[0].logprobs and response.choices[0].logprobs.content:
                top_tokens = response.choices[0].logprobs.content[0].top_logprobs
                prob_sim = 0.0001
                for token_obj in top_tokens:
                    token_str = token_obj.token.strip().lower()
                    if token_str in ['sim', 'yes', 's', 'y']:
                        prob_sim = math.exp(token_obj.logprob)
                        break
                probs.append(prob_sim)
            else:
                probs.append(0.5)
                
        except Exception:
            probs.append(0.5)
    
    # Calcular métricas ISR
    probs_array = np.array(probs)
    p_mean = np.mean(probs_array)
    p_min = np.min(probs_array)
    
    if p_min < 0.2:
        isr_decisao = "BLOQUEADO"
        motivo = f"Instabilidade detectada (P_min={p_min:.4f})"
    else:
        isr_decisao = "APROVADO"
        motivo = f"Confiança adequada (P_mean={p_mean:.4f})"
    
    # Calcular ISR
    target = 0.95
    epsilon = 1e-9
    
    if p_min > epsilon:
        b2t = np.log(target / max(p_min, 0.125))
        delta = np.mean([np.log(max(p_mean, epsilon) / max(p, epsilon)) for p in probs_array])
        isr = delta / max(b2t, epsilon) if b2t > 0 else 10.0
    else:
        isr = 0.0
        b2t = 999.0
        delta = 0.0
    
    return {
        "isr_decisao": isr_decisao,
        "motivo": motivo,
        "metrics": {
            "ISR": round(float(isr), 4),
            "B2T": round(float(b2t), 4),
            "Delta": round(float(delta), 4),
            "P_Mean": round(float(p_mean), 4),
            "P_Min": round(float(p_min), 4),
            "instabilidade": bool(p_min < 0.2)
        }
    }

print("[OK] Função de chamada ao modelo com ISR definida")

[OK] Função de chamada ao modelo com ISR definida


---
## 6. Executando Teste Blind

**IMPORTANTE:** O modelo recebe apenas dados anonimizados. Nenhuma pista sobre tipo ou decisão esperada.

In [11]:
# Preparar system prompt
SYSTEM_PROMPT_BLIND = criar_system_prompt_blind(POLITICAS_BANCO)

print(f"[OK] System prompt BLIND criado: {len(SYSTEM_PROMPT_BLIND)} caracteres")

[OK] System prompt BLIND criado: 20386 caracteres


In [12]:
# Selecionar casos para teste (balanceado por tipo)
casos_teste = []

for tipo in ["alucinacao", "score_baixo", "borderline", "bom_cliente", "multiplos_defaults"]:
    casos_tipo = [c for c in dataset_blind if c["ground_truth"]["tipo_caso"] == tipo]
    casos_teste.extend(casos_tipo[:6])  # 6 de cada tipo = 30 total

# Embaralhar para evitar padrões
random.shuffle(casos_teste)

print(f"[OK] Selecionados {len(casos_teste)} casos para teste BLIND")
print(f"\nDistribuição (interno):")
for tipo in ["alucinacao", "score_baixo", "borderline", "bom_cliente", "multiplos_defaults"]:
    count = sum(1 for c in casos_teste if c["ground_truth"]["tipo_caso"] == tipo)
    print(f"  - {tipo}: {count}")

[OK] Selecionados 22 casos para teste BLIND

Distribuição (interno):
  - alucinacao: 2
  - score_baixo: 6
  - borderline: 5
  - bom_cliente: 6
  - multiplos_defaults: 3


In [13]:
import time

# Executar testes
resultados_padrao = []
resultados_isr = []

print("Iniciando execução dos testes BLIND...")
print("="*70)
print("[INFO] Modelo recebe dados ANONIMIZADOS")
print("[INFO] Nenhuma pista sobre tipo ou decisão esperada")
print("="*70)

for i, item in enumerate(casos_teste):
    cliente_anonimo = item["cliente_anonimo"]
    cliente_original = item["cliente_original"]
    ground_truth = item["ground_truth"]
    
    esperado = ground_truth["decisao_esperada"]
    tipo_caso = ground_truth["tipo_caso"]
    
    # Log interno (não vai para o modelo)
    print(f"\n[{i+1}/{len(casos_teste)}] ID Anônimo: {cliente_anonimo['cliente_id']}")
    print(f"    Score: {cliente_anonimo.get('score_atual', 'N/A')}")
    
    # Modelo Padrão (recebe dados anonimizados)
    print(f"    -> Modelo padrão...", end=" ")
    resp_padrao = chamar_modelo_padrao_blind(cliente_anonimo, SYSTEM_PROMPT_BLIND)
    
    if resp_padrao["sucesso"]:
        decisao_padrao = resp_padrao["resposta_json"].get("decisao", "ERRO")
        print(f"{decisao_padrao}")
    else:
        decisao_padrao = "ERRO"
        print(f"ERRO")
    
    resultados_padrao.append({
        "cliente_id_anonimo": cliente_anonimo["cliente_id"],
        "cliente_id_original": cliente_original.get("cliente_id", "N/A"),
        "score": cliente_anonimo.get("score_atual"),
        "tipo_caso": tipo_caso,
        "esperado": esperado,
        "obtido": decisao_padrao,
        "eh_ficticio": ground_truth["eh_ficticio"],
        "acertou": decisao_padrao == esperado
    })
    
    time.sleep(1)
    
    # Agente ISR
    print(f"    -> Modelo com ISR...", end=" ")
    resp_isr = chamar_modelo_com_isr_blind(cliente_anonimo, cliente_original, SYSTEM_PROMPT_BLIND)
    
    if resp_isr["sucesso"]:
        decisao_isr = resp_isr["resposta_json"].get("decisao", "ERRO")
        isr_valor = resp_isr.get("isr_valor", 0)
        print(f"{decisao_isr} (ISR: {isr_valor:.4f})")
    else:
        decisao_isr = "ERRO"
        isr_valor = 0
        print(f"ERRO")
    
    resultados_isr.append({
        "cliente_id_anonimo": cliente_anonimo["cliente_id"],
        "cliente_id_original": cliente_original.get("cliente_id", "N/A"),
        "score": cliente_anonimo.get("score_atual"),
        "tipo_caso": tipo_caso,
        "esperado": esperado,
        "obtido": decisao_isr,
        "eh_ficticio": ground_truth["eh_ficticio"],
        "acertou": decisao_isr == esperado,
        "isr_valor": isr_valor,
        "isr_metrics": resp_isr.get("isr_metrics", {})
    })
    
    time.sleep(1)

print("\n" + "="*70)
print("[OK] Execução BLIND concluída!")

Iniciando execução dos testes BLIND...
[INFO] Modelo recebe dados ANONIMIZADOS
[INFO] Nenhuma pista sobre tipo ou decisão esperada

[1/22] ID Anônimo: CLI_71438261
    Score: 750
    -> Modelo padrão... APROVADA
    -> Modelo com ISR... APROVADA (ISR: 10.0000)

[2/22] ID Anônimo: CLI_C4077072
    Score: 820
    -> Modelo padrão... NEGADA
    -> Modelo com ISR... NEGADA (ISR: 0.0623)

[3/22] ID Anônimo: CLI_F0F254F9
    Score: 480
    -> Modelo padrão... NEGADA
    -> Modelo com ISR... NEGADA (ISR: 10.0000)

[4/22] ID Anônimo: CLI_FFC48D59
    Score: 620
    -> Modelo padrão... NEGADA
    -> Modelo com ISR... NEGADA (ISR: 0.0234)

[5/22] ID Anônimo: CLI_7337F832
    Score: 590
    -> Modelo padrão... NEGADA
    -> Modelo com ISR... NEGADA (ISR: 0.0000)

[6/22] ID Anônimo: CLI_3F64C458
    Score: 850
    -> Modelo padrão... APROVADA
    -> Modelo com ISR... APROVADA (ISR: 10.0000)

[7/22] ID Anônimo: CLI_D2FE6106
    Score: 780
    -> Modelo padrão... APROVADA
    -> Modelo com ISR... AP

---
## 7. Calculando Métricas

In [14]:
def calcular_metricas(resultados: List[Dict]) -> Dict[str, Any]:
    """Calcula métricas de ML."""
    
    def to_binary(decisao: str) -> int:
        return 1 if decisao == "APROVADA" else 0
    
    def normalizar(decisao: str) -> str:
        if decisao in ["NEGADA", "RECUSADA"]:
            return "NEGADA"
        return decisao
    
    valid_resultados = [r for r in resultados if r["obtido"] != "ERRO"]
    
    if not valid_resultados:
        return {"erro": "Nenhum resultado válido"}
    
    y_true = [to_binary(normalizar(r["esperado"])) for r in valid_resultados]
    y_pred = [to_binary(normalizar(r["obtido"])) for r in valid_resultados]
    
    tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
    tn = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 0)
    fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)
    fn = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 0)
    
    total = len(valid_resultados)
    
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    casos_ficticios = [r for r in valid_resultados if r["eh_ficticio"]]
    alucinacoes_detectadas = sum(1 for r in casos_ficticios if r["obtido"] in ["NEGADA", "RECUSADA"])
    taxa_deteccao_alucinacao = alucinacoes_detectadas / len(casos_ficticios) if casos_ficticios else 1.0
    
    return {
        "confusion_matrix": {"TP": int(tp), "TN": int(tn), "FP": int(fp), "FN": int(fn)},
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "false_positives": int(fp),
        "taxa_deteccao_alucinacao": float(taxa_deteccao_alucinacao),
        "total_ficticios": int(len(casos_ficticios)),
        "ficticios_detectados": int(alucinacoes_detectadas),
        "total_validos": int(total)
    }

metricas_padrao = calcular_metricas(resultados_padrao)
metricas_isr = calcular_metricas(resultados_isr)

print("[OK] Métricas calculadas!")

[OK] Métricas calculadas!


---
## 8. Resultados: Modelo Padrão (Blind)

In [15]:
print("="*70)
print("MODELO PADRÃO - BLIND TEST (sem pistas nos dados)")
print("="*70)

if "erro" not in metricas_padrao:
    cm = metricas_padrao["confusion_matrix"]
    print(f"\nMatriz de Confusão:")
    print(f"  +------------------+------------------+")
    print(f"  |  TP = {cm['TP']:>3}       |  FN = {cm['FN']:>3}       |")
    print(f"  | (Aprovou certo)  | (Perdeu cliente) |")
    print(f"  +------------------+------------------+")
    print(f"  |  FP = {cm['FP']:>3}       |  TN = {cm['TN']:>3}       |")
    print(f"  | (RISCO!)         | (Negou certo)    |")
    print(f"  +------------------+------------------+")
    
    print(f"\nMétricas:")
    print(f"  Accuracy:  {metricas_padrao['accuracy']:.1%}")
    print(f"  Precision: {metricas_padrao['precision']:.1%}")
    print(f"  Recall:    {metricas_padrao['recall']:.1%}")
    print(f"  F1-Score:  {metricas_padrao['f1_score']:.1%}")
    
    print(f"\n[CRÍTICO] Detecção de Alucinação (sem pistas):")
    print(f"  Clientes fictícios: {metricas_padrao['total_ficticios']}")
    print(f"  Detectados: {metricas_padrao['ficticios_detectados']}")
    print(f"  Taxa de detecção: {metricas_padrao['taxa_deteccao_alucinacao']:.1%}")
    
    if metricas_padrao["false_positives"] > 0:
        print(f"\n[ALERTA] {metricas_padrao['false_positives']} APROVAÇÕES INDEVIDAS!")
else:
    print(f"ERRO: {metricas_padrao['erro']}")

MODELO PADRÃO - BLIND TEST (sem pistas nos dados)

Matriz de Confusão:
  +------------------+------------------+
  |  TP =   6       |  FN =   0       |
  | (Aprovou certo)  | (Perdeu cliente) |
  +------------------+------------------+
  |  FP =   0       |  TN =  16       |
  | (RISCO!)         | (Negou certo)    |
  +------------------+------------------+

Métricas:
  Accuracy:  100.0%
  Precision: 100.0%
  Recall:    100.0%
  F1-Score:  100.0%

[CRÍTICO] Detecção de Alucinação (sem pistas):
  Clientes fictícios: 2
  Detectados: 2
  Taxa de detecção: 100.0%


---
## 9. Resultados: Agente ISR (Blind)

In [16]:
print("="*70)
print("AGENTE COM ISR - BLIND TEST")
print("="*70)

if "erro" not in metricas_isr:
    cm = metricas_isr["confusion_matrix"]
    print(f"\nMatriz de Confusão:")
    print(f"  +------------------+------------------+")
    print(f"  |  TP = {cm['TP']:>3}       |  FN = {cm['FN']:>3}       |")
    print(f"  | (Aprovou certo)  | (Perdeu cliente) |")
    print(f"  +------------------+------------------+")
    print(f"  |  FP = {cm['FP']:>3}       |  TN = {cm['TN']:>3}       |")
    print(f"  | (RISCO!)         | (Negou certo)    |")
    print(f"  +------------------+------------------+")
    
    print(f"\nMétricas:")
    print(f"  Accuracy:  {metricas_isr['accuracy']:.1%}")
    print(f"  Precision: {metricas_isr['precision']:.1%}")
    print(f"  Recall:    {metricas_isr['recall']:.1%}")
    print(f"  F1-Score:  {metricas_isr['f1_score']:.1%}")
    
    print(f"\n[PROTEÇÃO] Detecção de Alucinação:")
    print(f"  Clientes fictícios: {metricas_isr['total_ficticios']}")
    print(f"  Detectados: {metricas_isr['ficticios_detectados']}")
    print(f"  Taxa de detecção: {metricas_isr['taxa_deteccao_alucinacao']:.1%}")
    
    if metricas_isr["false_positives"] == 0:
        print(f"\n[SUCESSO] ZERO aprovações indevidas!")
    else:
        print(f"\n[ATENÇÃO] {metricas_isr['false_positives']} aprovações indevidas")
else:
    print(f"ERRO: {metricas_isr['erro']}")

AGENTE COM ISR - BLIND TEST

Matriz de Confusão:
  +------------------+------------------+
  |  TP =   6       |  FN =   0       |
  | (Aprovou certo)  | (Perdeu cliente) |
  +------------------+------------------+
  |  FP =   0       |  TN =  16       |
  | (RISCO!)         | (Negou certo)    |
  +------------------+------------------+

Métricas:
  Accuracy:  100.0%
  Precision: 100.0%
  Recall:    100.0%
  F1-Score:  100.0%

[PROTEÇÃO] Detecção de Alucinação:
  Clientes fictícios: 2
  Detectados: 2
  Taxa de detecção: 100.0%

[SUCESSO] ZERO aprovações indevidas!


---
## 10. Comparação Final (Blind Test)

In [17]:
print("="*80)
print("COMPARAÇÃO BLIND TEST: MODELO PADRÃO vs AGENTE ISR")
print("="*80)
print("\n[INFO] Dados ANONIMIZADOS - Sem vazamento de informação")
print("[INFO] O modelo NÃO recebeu pistas sobre tipo ou decisão esperada\n")

if "erro" not in metricas_padrao and "erro" not in metricas_isr:
    print(f"{'Métrica':<30} {'Modelo Padrão':>15} {'Agente ISR':>15} {'Diferença':>15}")
    print("-"*80)
    
    diff_acc = metricas_isr['accuracy'] - metricas_padrao['accuracy']
    print(f"{'Accuracy':<30} {metricas_padrao['accuracy']:>14.1%} {metricas_isr['accuracy']:>14.1%} {diff_acc:>+14.1%}")
    
    diff_prec = metricas_isr['precision'] - metricas_padrao['precision']
    print(f"{'Precision':<30} {metricas_padrao['precision']:>14.1%} {metricas_isr['precision']:>14.1%} {diff_prec:>+14.1%}")
    
    diff_rec = metricas_isr['recall'] - metricas_padrao['recall']
    print(f"{'Recall':<30} {metricas_padrao['recall']:>14.1%} {metricas_isr['recall']:>14.1%} {diff_rec:>+14.1%}")
    
    diff_f1 = metricas_isr['f1_score'] - metricas_padrao['f1_score']
    print(f"{'F1-Score':<30} {metricas_padrao['f1_score']:>14.1%} {metricas_isr['f1_score']:>14.1%} {diff_f1:>+14.1%}")
    
    print("-"*80)
    
    diff_fp = metricas_isr['false_positives'] - metricas_padrao['false_positives']
    print(f"{'False Positives (RISCO!)':<30} {metricas_padrao['false_positives']:>15} {metricas_isr['false_positives']:>15} {diff_fp:>+15}")
    
    diff_aluc = metricas_isr['taxa_deteccao_alucinacao'] - metricas_padrao['taxa_deteccao_alucinacao']
    print(f"{'Detecção de Alucinação':<30} {metricas_padrao['taxa_deteccao_alucinacao']:>14.1%} {metricas_isr['taxa_deteccao_alucinacao']:>14.1%} {diff_aluc:>+14.1%}")
    
    print("="*80)
else:
    print("Erro ao calcular métricas.")

COMPARAÇÃO BLIND TEST: MODELO PADRÃO vs AGENTE ISR

[INFO] Dados ANONIMIZADOS - Sem vazamento de informação
[INFO] O modelo NÃO recebeu pistas sobre tipo ou decisão esperada

Métrica                          Modelo Padrão      Agente ISR       Diferença
--------------------------------------------------------------------------------
Accuracy                               100.0%         100.0%          +0.0%
Precision                              100.0%         100.0%          +0.0%
Recall                                 100.0%         100.0%          +0.0%
F1-Score                               100.0%         100.0%          +0.0%
--------------------------------------------------------------------------------
False Positives (RISCO!)                     0               0              +0
Detecção de Alucinação                 100.0%         100.0%          +0.0%


---
## 11. Análise Detalhada

In [18]:
# Mostrar casos onde o modelo errou (sem ter pistas)
print("="*80)
print("ANÁLISE: ERROS DO MODELO PADRÃO (sem pistas)")
print("="*80)

erros_padrao = [r for r in resultados_padrao if not r["acertou"]]

if erros_padrao:
    print(f"\nTotal de erros: {len(erros_padrao)}")
    print("\nDetalhamento:")
    
    for erro in erros_padrao[:10]:  # Mostrar até 10
        print(f"\n  ID Original: {erro['cliente_id_original']}")
        print(f"  ID Anônimo: {erro['cliente_id_anonimo']}")
        print(f"  Tipo (interno): {erro['tipo_caso']}")
        print(f"  Score: {erro['score']}")
        print(f"  Esperado: {erro['esperado']} | Obtido: {erro['obtido']}")
        if erro['eh_ficticio']:
            print(f"  [CRÍTICO] Era cliente fictício e foi aprovado!")
else:
    print("\nNenhum erro do modelo padrão!")

ANÁLISE: ERROS DO MODELO PADRÃO (sem pistas)

Total de erros: 1

Detalhamento:

  ID Original: TEST_BORDERLINE_001
  ID Anônimo: CLI_FFC48D59
  Tipo (interno): borderline
  Score: 620
  Esperado: ANALISE_GERENCIAL | Obtido: NEGADA


In [19]:
# Casos onde ISR fez diferença
print("\n" + "="*80)
print("CASOS ONDE ISR FEZ DIFERENÇA")
print("="*80)

diferenca_casos = []
for rp, ri in zip(resultados_padrao, resultados_isr):
    if rp["obtido"] != ri["obtido"]:
        diferenca_casos.append({
            "cliente_id_original": rp["cliente_id_original"],
            "tipo_caso": rp["tipo_caso"],
            "esperado": rp["esperado"],
            "padrao": rp["obtido"],
            "isr": ri["obtido"],
            "padrao_acertou": rp["acertou"],
            "isr_acertou": ri["acertou"],
            "eh_ficticio": rp["eh_ficticio"]
        })

if diferenca_casos:
    for caso in diferenca_casos:
        print(f"\nCliente: {caso['cliente_id_original']}")
        print(f"  Tipo: {caso['tipo_caso']}")
        print(f"  Esperado: {caso['esperado']}")
        print(f"  Padrão:   {caso['padrao']} ({'OK' if caso['padrao_acertou'] else 'ERRO'})")
        print(f"  ISR:      {caso['isr']} ({'OK' if caso['isr_acertou'] else 'ERRO'})")
        
        if caso['eh_ficticio'] and caso['isr_acertou'] and not caso['padrao_acertou']:
            print(f"  [ISR PROTEGEU] Bloqueou cliente fictício!")
else:
    print("\nNenhuma diferença entre os modelos.")


CASOS ONDE ISR FEZ DIFERENÇA

Cliente: TEST_BORDERLINE_002
  Tipo: borderline
  Esperado: ANALISE_GERENCIAL
  Padrão:   ANALISE_GERENCIAL (OK)
  ISR:      NEGADA (ERRO)


---
## 12. Salvando Resultados

In [ ]:
# Salvar resultados
output_dir = PROJECT_ROOT / "outputs" / "notebook_results"
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"comparacao_blind_test_{timestamp}.json"

def limpar_para_json(resultados):
    limpos = []
    for r in resultados:
        limpo = {}
        for k, v in r.items():
            if k == "resposta_completa":
                continue
            if hasattr(v, 'item'):
                limpo[k] = v.item()
            elif isinstance(v, bool):
                limpo[k] = bool(v)
            elif isinstance(v, dict):
                limpo[k] = {kk: (vv.item() if hasattr(vv, 'item') else bool(vv) if isinstance(vv, (bool, type(None))) else vv) for kk, vv in v.items()}
            else:
                limpo[k] = v
        limpos.append(limpo)
    return limpos

dados_salvar = {
    "metadata": {
        "timestamp": timestamp,
        "modelo": MODEL_NAME,
        "versao": "4.0-blind-test",
        "total_casos": len(casos_teste),
        "descricao": "Teste cego: dados anonimizados, sem vazamento de informação"
    },
    "resultados_padrao": limpar_para_json(resultados_padrao),
    "resultados_isr": limpar_para_json(resultados_isr),
    "metricas_padrao": metricas_padrao,
    "metricas_isr": metricas_isr
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(dados_salvar, f, indent=2, ensure_ascii=False)

print(f"[OK] Resultados salvos em: {output_file}")

---
## 13. Conclusões (Blind Test)

### O que esta versão testa

| Aspecto | Versão 3 (Zero-Shot) | Versão 4 (Blind Test) |
|---------|---------------------|----------------------|
| Cliente ID | `TEMP_ALUCINACAO_001` | `CLI_A3F2B1C4` |
| CPF | `999.999.999-99` | CPF válido gerado |
| Tipo no log | Visível | Apenas interno |
| Data leakage | Possível | Eliminado |

### Interpretação dos Resultados

1. **Se performance caiu muito**: O modelo na v3 estava usando as pistas (data leakage)
2. **Se performance similar**: O modelo realmente analisa dados financeiros
3. **ISR continua útil**: Detecta inconsistências nos dados originais

### Recomendações

1. **Use sempre blind test** para avaliações justas
2. **Compare v3 vs v4** para medir o impacto do data leakage
3. **ISR detecta por dados** não por pistas nos identificadores